# PEL218 - Atividade 1
Processamento de Linguagem Narual (Prof. Guilherme Wachs)

Claudio Aparecido Borges Junior (RA 120122-7)

## Atividade
Criar um gerador de valores monetários por extenso a partir de uma entrada numérica (de 0 até 999) usando um FST.

## Resolução
Foi escolhida a linguagem *Python* versão 3.8.5. A matriz de transição foi modelada como um dicionário

In [1]:
from collections import defaultdict
t = defaultdict(dict)

In [7]:
# All possible units
units = [
    'zero',
    'um',
    'dois',
    'três',
    'quatro',
    'cinco',
    'seis',
    'sete',
    'oito',
    'nove',
]

# All possible dozens starting with 1
dozen1x = [
    'dez',
    'onze',
    'doze',
    'treze',
    'quatorze',
    'quinze',
    'dezesseis',
    'dezesete',
    'dezoito',
    'dezenove',
]

# All possible dozens that do not start with 1
dozen2p = [
    '', # Not used
    '', # Not used
    'vinte',
    'trinta',
    'quarenta',
    'cinquenta',
    'sessenta',
    'setenta',
    'oitenta',
    'noventa',
]

# All possible hundred that has the last element equal to 0
hundredsxy0 = [
    '', # Not used
    'cem',
    'duzentos',
    'trezentos',
    'quatrocentos',
    'quinhentos',
    'seiscentos',
    'setecentos',
    'oitocentos',
    'novecentos',
]

# All possible hundred that has the last element not equal to 0
hundreds = [
    '', # Not used
    'cento',
    'duzentos',
    'trezentos',
    'quatrocentos',
    'quinhentos',
    'seiscentos',
    'setecentos',
    'oitocentos',
    'novecentos',
]

A very long transition table was created. The table contains a valid state for each possible output and it expects a '\n' as a final marker to identify the end of elements

In [3]:
# Define the final state
t['qf'] = {}
# Keep reading left zeros until there is no more
t['qi'].update({'0': ('', 'qi'), '\n': ('zero', 'qf')})
# Create a state for each first element different than 0 and \n
t['qi'].update({str(i): ('', 'q' + str(i)) for i in range(1, 10)})
# If there is only one element, print a single number
for i in range(1, 10):
    # Trivial solution for 1 element
    statex = 'q' + str(i)
    t[statex].update({'\n': (units[i], 'qf')})
    for j in range(10):
        state1x = 'q' + str(i * 10 + j)
        t[statex].update({str(j): ('', state1x)})

        if i == 1:
            # 10 until 19; they have special names
            t[state1x].update({'\n': (dozen1x[j], 'qf')})
        elif j == 0:
            # x0 untill y0, they don't have the ' e '
            t[state1x].update({'\n': (dozen2p[i], 'qf')})
        else:
            # Regular use case for 2 elements
            t[state1x].update({'\n': (dozen2p[i] + ' e ' + units[j], 'qf')})
        
        for z in range(10):
            state_1xx = 'q' + str(i * 100 + j * 10 + z)

            # In case the value is grether than 100
            t[state1x].update({str(z): ('', state_1xx)})

        
            if j == 0 and z == 0:
                # Special condition for both unit and dozen equal to 0
                t[state_1xx].update({'\n': (hundredsxy0[i], 'qf')})
            elif j == 0:
                # Special condition for dozen equal to 0
                t[state_1xx].update({'\n': (hundreds[i] + ' e ' + units[z], 'qf')})
            elif j == 1:
                # Special condition for dozen equal to 1
                t[state_1xx].update({'\n': (hundreds[i] + ' e ' + dozen1x[z], 'qf')})
            elif z == 0:
                # Special condition for unit equal to 0
                t[state_1xx].update({'\n': (hundreds[i] + ' e ' + dozen2p[j], 'qf')})
            else:
                # All other conditions
                t[state_1xx].update({'\n': (hundreds[i] + ' e ' + dozen2p[j] + ' e ' + units[z], 'qf')})

Let's define the parser. It's a initial and final states, a transiction table and a list of elements representing the input. For each element, identify a valid transition and execute it. Accumulate the output in an array. Terminate if all elements have been parsed correctly and the final state has been reached.

In [4]:
def parse(q_ini, q_end, transitions, elements):
    output = []
    q_curr = q_ini
    for c in elements:
        if q_curr not in transitions:
            # Undefined condition
            return None
    
        if c not in transitions[q_curr]:
            # Illegal state
            return None
        
        partial_output, q_curr, = transitions[q_curr][c]
        output.append(partial_output)
    
    if q_curr != q_end:
        # Not a valid solution
        return None
    return output

To test the solution, print all elements from 0 until 999.

In [5]:
# Print all conditions
for i in range(0, 1000):
    print(''.join(parse('qi', 'qf', t, str(i) + '\n')))

zero
um
dois
três
quatro
cinco
seis
sete
oito
nove
dez
onze
doze
treze
quatorze
quinze
dezesseis
dezesete
dezoito
dezenove
vinte
vinte e um
vinte e dois
vinte e três
vinte e quatro
vinte e cinco
vinte e seis
vinte e sete
vinte e oito
vinte e nove
trinta
trinta e um
trinta e dois
trinta e três
trinta e quatro
trinta e cinco
trinta e seis
trinta e sete
trinta e oito
trinta e nove
quarenta
quarenta e um
quarenta e dois
quarenta e três
quarenta e quatro
quarenta e cinco
quarenta e seis
quarenta e sete
quarenta e oito
quarenta e nove
cinquenta
cinquenta e um
cinquenta e dois
cinquenta e três
cinquenta e quatro
cinquenta e cinco
cinquenta e seis
cinquenta e sete
cinquenta e oito
cinquenta e nove
sessenta
sessenta e um
sessenta e dois
sessenta e três
sessenta e quatro
sessenta e cinco
sessenta e seis
sessenta e sete
sessenta e oito
sessenta e nove
setenta
setenta e um
setenta e dois
setenta e três
setenta e quatro
setenta e cinco
setenta e seis
setenta e sete
setenta e oito
setenta e nove
oit

Let's get some numbers about the transition table:

In [6]:
print('Number of state: ', len(t))
print('Maximum number of first level transitions :', max([len(e) for e in t.values()]))

Number of state:  1001
Maximum number of first level transitions : 11
